# Hard-to-count reservations analysis

By [Ben Welsh](https://palewi.re/who-is-ben-welsh/) and [Sandhya Kambhampati]()

The analysis was conducted for the June X, 2019, Los Angeles Times story [Headline TK](). 

It found that as the 2020 census nears, concerns about an undercount of Native Americans is gaining traction here and across the country. The finding was based, in part, on a review of public data, as well as some original analysis by The Times. 

The finding from our research:

* Nearly 600,000 Native Americans live on tribal reservations, semi-sovereign entities governed by elected indigenous leaders. On the Navajo Nation — the country’s largest reservation — 175,000 people live in a mostly rural high desert area bigger than West Virginia.

* During the 2010 count, one out of every seven Native Americans living on a reservation was missed, according to an audit by the U.S. Census Bureau. That adds up to 82,000 people overlooked and uncounted — equal to skipping the entire city of Santa Fe, New Mexico’s capital.

* The Census Bureau’s struggles in 2010 resulted in more than 80% of reservation lands being ranked among the country’s hardest-to-count areas, according to a Times analysis of estimates published by the City University of New York.

Here's how we arrived at each conclusion.

### Import Python tools

The tools we used to do our work.

In [3]:
import pandas as pd
import geopandas as gpd

Utilities to download Census data from the Data Desk's [census-data-downloader](https://github.com/datadesk/census-data-downloader/)

In [11]:
read_acs = lambda x: pd.read_csv(
    f"https://raw.githubusercontent.com/datadesk/census-data-downloader/ba0ca0fcf30ebcaf7a42d2babbccb1b057312e3f/data/processed/acs5_2017_{x}.csv",
    dtype={"geoid": str}
)

### The Native American population

After the 2010 count, the Census Bureau conducted an internal audit known as the "Coverage Measurement Analysis." It concluded that there are roughly 600,000 Native Americans living on a federal reservations. This number was confirmed by a press office at the bureau.

In [ ]:
# DocumentCloud embed goes here

Navajo Nation's total population, roughly 175,000, is available from the bureau's American Community Survey.

In [13]:
raw_homeland_population = read_acs("population_aiannhhomeland")

In [14]:
raw_homeland_population[raw_homeland_population.name.str.contains("Navajo Nation")]

,geoid,name,universe,american indian area/alaska native area/hawaiian home land
175,2430,Navajo Nation Reservation and Off-Reservation ...,175005.0,2430


### The Native American undercount

The "Coverage Measurement Analysis" also estimated the the undercount of Native Americans on reservations. It found a statistically significant net 5% undercount, which includes people counted twice, people imputed entirely by statistics and people missed entirely. The total number of people missed, known as the omission rate, was 13.8%. This number was confirmed by the press office at the bureau.

### The hard-to-count areas

#### Hard to count tracts

Import every Census tract with its hard-to-count status according to the [Center for Urban Research at the City University of New York Graduate Center](https://www.censushardtocountmaps2020.us/).

In [3]:
tracts = gpd.read_file("data/analysis/hard-to-count-tracts.geojson")

In [4]:
tracts.head()

,geoid,state,county,pop,mrr,mrr_htc,ue,htc,geometry
0,01005950300,Alabama,Barbour County,1813,77.9,0,0,0,POLYGON ((-85.52744143500958 31.86650898512998...
1,01005950900,Alabama,Barbour County,3888,74.6,0,0,0,POLYGON ((-85.16412632985951 31.83060098962552...
2,01005950800,Alabama,Barbour County,2157,83.5,0,0,0,POLYGON ((-85.14872232914963 31.90934100751823...
3,01005950700,Alabama,Barbour County,1775,79.6,0,0,0,POLYGON ((-85.14578832747586 31.89149700368442...
4,01005950600,Alabama,Barbour County,2120,79.4,0,0,0,"POLYGON ((-85.14572732739356 31.8901120033811,..."


How many Census tracts are there total?

In [4]:
len(tracts)

72845

Filter down to just the hard to count tracts

In [6]:
htc = tracts[tracts.htc >= 1]

How many hard to county tracts are there?

In [7]:
len(htc)

14800

Write that out to a SHP

In [8]:
htc.to_file("./data/analysis/htc-tracts.shp")

#### Tribal reservations

Read in tribal homelands from the [U.S. Census Bureau's TIGER/Line Shapefiles service](https://www.census.gov/cgi-bin/geo/shapefiles/index.php).

In [9]:
raw_homelands = gpd.read_file("data/tiger/tl_2018_us_aiannh/tl_2018_us_aiannh.shp")

In [ ]:
raw_homelands.head()

Clean that up.

In [10]:
trimmed_homelands = raw_homelands[['AIANNHCE', 'NAMELSAD', 'geometry']]

In [11]:
prepped_homelands = trimmed_homelands.rename(columns={
    "AIANNHCE": "geoid",
    "NAMELSAD": "name",
})

In [12]:
prepped_homelands.head()

,geoid,name,geometry
0,2320,Mohegan Reservation,"(POLYGON ((-72.089125 41.480395, -72.089004 41..."
1,9100,Golden Hill Paugussett (state) Reservation,"(POLYGON ((-72.26917299999999 41.554725, -72.2..."
2,4110,Table Mountain Rancheria,"POLYGON ((-119.640915 36.984246, -119.640807 3..."
3,1380,Greenville Rancheria,"POLYGON ((-120.897365 40.150998, -120.897334 4..."
4,1110,Flathead Reservation,"POLYGON ((-114.851209 47.887207, -114.819894 4..."


The homelands come in a variety of types. Only a subset are the federally-recognized resevations we want to study.

In [6]:
def get_geotype(geoid):
    """
    Accepts the GEOID of a American Indian, Alaska Native or Native Hawaiian homeland.
    
    Returns which type of homeland it is.
    
    They are defined on pages 47 and 48 of this Census Bureau document:
    
        https://www.census.gov/prod/cen2010/doc/pl94-171.pdf
    """
    val = int(geoid)
    if val < 5000:
        return 'Federal reservation or trust land'
    elif val < 5500:
        return 'Hawaiian home land'
    elif val < 6000:
        return 'Oklahoma tribal statistical area'
    elif val < 8000:
        return 'Alaska native village statistical area'
    elif val < 9000:
        return 'Tribal-designated statistical area'
    elif val < 9500:
        return 'State reservation'
    elif val < 9999:
        return 'State-designated tribal statistical area'

In [15]:
prepped_tribal['geotype'] = prepped_tribal.geoid.apply(get_geotype)

In [16]:
prepped_tribal.geotype.value_counts(dropna=False)

Federal reservation or trust land           477
Alaska native village statistical area      220
Hawaiian home land                           75
State-designated tribal statistical area     30
Oklahoma tribal statistical area             29
State reservation                            10
Tribal-designated statistical area            4
Name: geotype, dtype: int64

In [18]:
prepped_tribal.sort_values("name").head(10)

,geoid,name,geometry,geotype
476,0010,Acoma Off-Reservation Trust Land,"(POLYGON ((-107.606714 34.454814, -107.605615 ...",Federal reservation or trust land
308,0010,Acoma Pueblo,"(POLYGON ((-107.558965 35.071588, -107.558965 ...",Federal reservation or trust land
814,9510,Adais Caddo SDTSA,"(POLYGON ((-93.013549 31.563087, -93.013505999...",State-designated tribal statistical area
246,0020,Agua Caliente Indian Reservation,"(POLYGON ((-116.423049 33.807703, -116.423014 ...",Federal reservation or trust land
442,0020,Agua Caliente Off-Reservation Trust Land,"(POLYGON ((-116.486361 33.791076, -116.486106 ...",Federal reservation or trust land
571,6015,Akhiok ANVSA,"POLYGON ((-154.266866 56.962027, -154.264854 5...",Alaska native village statistical area
764,6020,Akiachak ANVSA,"POLYGON ((-161.487366 60.881786, -161.487158 6...",Alaska native village statistical area
625,6025,Akiak ANVSA,"POLYGON ((-161.23455 60.920581, -161.234417 60...",Alaska native village statistical area
565,6030,Akutan ANVSA,"POLYGON ((-165.808239 54.139897, -165.786582 5...",Alaska native village statistical area
405,0050,Alabama-Coushatta Off-Reservation Trust Land,"(POLYGON ((-94.666544 30.723948, -94.666355999...",Federal reservation or trust land


Filter down to just the reservations.

In [25]:
reservations = prepped_tribal[prepped_tribal.geotype == 'Federal reservation or trust land']

Dissolve reservations and trust lands into single shape using their common unique identifier.

In [26]:
reservations_deduped = reservations.dissolve(by='geoid').reset_index()

Run the count again.

In [17]:
reservations_deduped.geotype.value_counts(dropna=False)

NameError: name 'reservations_deduped' is not defined

Calculate the total area of each reservation.

In [28]:
reservations_deduped['area'] = reservations_deduped.to_crs({'proj':'cea'}).geometry.map(lambda p: p.area / 10**6)

Show the biggest ones.

In [29]:
reservations_deduped.sort_values("area", ascending=False).head()

,geoid,geometry,name,geotype,area
173,2430,"(POLYGON ((-107.911233 36.379858, -107.911205 ...",Navajo Nation Reservation,Federal reservation or trust land,62565.921391
296,4390,"(POLYGON ((-111.098505 39.589313, -111.098459 ...",Uintah and Ouray Reservation,Federal reservation or trust land,17674.267493
284,4200,"(POLYGON ((-111.217214 32.103731, -111.217158 ...",Tohono O'odham Nation Reservation,Federal reservation or trust land,11534.986075
40,0605,"(POLYGON ((-100.586764 44.226201, -100.584494 ...",Cheyenne River Reservation,Federal reservation or trust land,11445.301110
198,2810,"POLYGON ((-103.001016 43.600805, -103.001016 4...",Pine Ridge Reservation,Federal reservation or trust land,11275.048911


#### Geospatial analysis

Reproject the reservations to match the tracts.

In [32]:
reservations_reprojected = reservations_deduped.to_crs(htc.crs)

Return the areas that intersect between the reservations and the hard-to-count tracts.

In [33]:
intersection = gpd.overlay(reservations_reprojected, htc, how='intersection')

How many intersections are there?

In [35]:
len(intersection)

553

Dissolve the intersecting tracts using each reservations unique identifier.

In [36]:
dissolved = intersection.dissolve(by='geoid_1')

How many reservations intersect with a hard-to-count tract?

In [37]:
len(dissolved)

170

Clean up the dataframe.

In [39]:
dissolved_trimmed = dissolved.reset_index()[[
    'geoid_1',
    'name',
    'geometry'
]]

In [40]:
dissolved_prepped = dissolved_trimmed.rename(columns={
    "geoid_1": "geoid",
})

Calculate the total area of each intersection.

In [42]:
dissolved_prepped['area'] = dissolved_prepped.to_crs({'proj':'cea'}).geometry.map(lambda p: p.area / 10**6)

Merge intersections with the reservations.

In [44]:
dissolved_merged = dissolved_prepped.merge(
    reservations_deduped[['geoid', 'area']],
    on="geoid",
    suffixes=["_htc", "_total"]
)

Calculate the percent of each reservation's area that intersects with a hard-to-count tract.

In [46]:
dissolved_merged['pct_htc'] = dissolved_merged.area_htc / dissolved_merged.area_total

Show the largest percentages.

In [47]:
dissolved_merged.sort_values("area_htc", ascending=False).head(5)

,geoid,name,geometry,area_htc,area_total,pct_htc
88,2430,Navajo Nation Reservation,"(POLYGON ((-108.344659 34.711661, -108.3446588...",62315.774112,62565.921391,0.996002
151,4390,Uintah and Ouray Reservation,POLYGON ((-110.0241408869228 39.46170078192419...,14963.114044,17674.267493,0.846604
144,4200,Tohono O'odham Nation Reservation,"(POLYGON ((-112.719808 32.967355, -112.719534 ...",11534.902203,11534.986075,0.999993
18,0605,Cheyenne River Reservation,POLYGON ((-101.8710226103383 44.53181420775375...,11424.555187,11445.301110,0.998187
139,3970,Standing Rock Reservation,POLYGON ((-101.4702072145578 45.47181753760831...,9486.074003,9486.191063,0.999988


Show the smallest percentages.

In [48]:
dissolved_merged.sort_values("area_htc", ascending=False).tail(5)

,geoid,name,geometry,area_htc,area_total,pct_htc
99,2715,Pauma and Yuima Reservation,(POLYGON ((-116.9321159667102 33.3464051059000...,0.007937,24.252290,3.272689e-04
67,1850,La Jolla Reservation,(POLYGON ((-116.8410570065867 33.2559419054459...,0.002918,34.957758,8.348507e-05
15,0495,Capitan Grande Reservation,POLYGON ((-116.6811140223018 32.88451265870786...,0.002156,64.430007,3.346085e-05
148,4315,Tunica-Biloxi Reservation,(POLYGON ((-92.0626268708872 31.10577978474263...,0.000030,3.149779,9.520608e-06
96,2635,Pala Reservation,"POLYGON ((-117.000974922776 33.38656981010941,...",0.000006,52.706195,1.098324e-07


Write out the result to a GeoJSON file.

In [66]:
dissolved_merged.to_file("data/analysis/htc-reservations.geojson", driver="GeoJSON")

How much total reservation land is hard to count?

In [50]:
dissolved_merged.area_htc.sum()

239727.28813155653

How much total reservation land is there overall?

In [51]:
reservations_deduped['area'].sum()

295020.57328184653

What percentage of reservation land is hard to count?

In [52]:
dissolved_merged.area_htc.sum() / reservations_deduped['area'].sum()

0.8125782058681521